In [ ]:
# numpy
import numpy as np

# pandas stuff
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# plotting stuff
from pandas.plotting import lag_plot
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
colorMap = sns.light_palette("blue", as_cmap=True)
#plt.rcParams.update({'font.size': 12})


# install dabl
!pip install dabl > /dev/null
import dabl
# install datatable
!pip install datatable > /dev/null
import datatable as dt

# misc
import missingno as msno

# system
import warnings
warnings.filterwarnings('ignore')
# for the image import
import os
from IPython.display import Image
# garbage collector to keep RAM in check
import gc  

In [ ]:
%%time

#利用datatable載入data

#train_data_datatable = dt.fread('../input/jane-street-market-prediction/train.csv')



In [ ]:
%%time
#再轉為pandas方式  載入時間縮短很多！！

#train_data = train_data_datatable.to_pandas()

In [ ]:
def data_explore_old(df):
    # 複製data
    data = df.copy()
    # 查看列名
    print(data.columns)
    # 查看data
    print(data.head())
    # 有空值
    print(data.sum())
    # 看平均數
    print(data.mean())
    print(data.describe())
    # 查看空值
    print(data.isnull().sum())
    # 最多的有1734個NAN，接近20%
    fig = plt.figure()
    fig, axes = plt.subplots(4, 2, sharex = True)
    for i in range(4):
        for j in range(2):
            pos = 2*i + j
            if pos > 6:
                break
            axes[i][j].set_title(data.columns[pos])
            axes[i][j].plot(data.iloc[:, pos])
    plt.subplots_adjust(wspace = 0.2, hspace = 1)
    plt.savefig("./output/targets_line.png")
    plt.close()
    # 畫特徵
    fig = plt.figure(figsize = (10, 80))
    for i in range(130):
        ax = fig.add_subplot(65, 2, i+1)
        ax.set_title(data.columns[i+7])
        plt.plot(data.iloc[:, i+7])
    plt.subplots_adjust(wspace = 0.2, hspace = 1)
    plt.savefig("./output/features_line.png")
    plt.close()
    
    fig = plt.figure()
    sns.distplot(data.iloc[:, 1:8], hist = True, bins = 100, kde = True)
    # data.iloc[:, 1:8].plot.hist(subplots = True, sharex = True, layout = (4, 2), bins = 50)
    plt.savefig("./output/targets_hist.png")
    # 化特徵
    fig = plt.figure()
    sns.distplot(data.iloc[:, 8:-2], hist = True, bins = 100, kde = True)
    # data.iloc[:, 8:-2].plot.hist(subplots = True, sharex = True, layout = (65, 2), figsize = (10, 80), bins = 50)
    plt.savefig("./output/features_hist.png")

In [ ]:
#顯示上面def要顯示的東西
#data_explore_old(train_data)  #記憶體不足 不顯示

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
balance= pd.Series(train_data['resp']).cumsum()
ax.set_xlabel ("Trade", fontsize=18)
ax.set_ylabel ("Cumulative resp", fontsize=18);
balance.plot(lw=3);
del balance
gc.collect();  #resp累積值

In [ ]:
'''fig, ax = plt.subplots(figsize=(15, 5))
balance= pd.Series(train_data['resp']).cumsum()
resp_1= pd.Series(train_data['resp_1']).cumsum()
resp_2= pd.Series(train_data['resp_2']).cumsum()
resp_3= pd.Series(train_data['resp_3']).cumsum()
resp_4= pd.Series(train_data['resp_4']).cumsum()
ax.set_xlabel ("Trade", fontsize=18)
ax.set_title ("Cumulative resp and time horizons 1, 2, 3, and 4 (500 days)", fontsize=18)
balance.plot(lw=3)
resp_1.plot(lw=3)
resp_2.plot(lw=3)
resp_3.plot(lw=3)
resp_4.plot(lw=3)
plt.legend(loc="upper left");
del resp_1
del resp_2
del resp_3
del resp_4
gc.collect();'''   #resp 1~4 累積圖

In [ ]:
percent_zeros = (100/train_data.shape[0])*((train_data.weight.values == 0).sum())
print('Percentage of zero weights is: %i' % percent_zeros +"%")
#17%的weights為0 處理時將0拿掉
min_weight = train_data['weight'].min()
print('The minimum weight is: %.2f' % min_weight)
#下面的確認沒有-的weights
max_weight = train_data['weight'].max()
print('The maximum weight was: %.2f' % max_weight)
#最大weight為167.27


In [ ]:
'''plt.figure(figsize = (12,5))
ax = sns.distplot(train_data['weight'], 
             bins=1400, 
             kde_kws={"clip":(0.001,1.4)}, 
             hist_kws={"range":(0.001,1.4)},
             color='darkcyan', 
             kde=False);
values = np.array([rec.get_height() for rec in ax.patches])
norm = plt.Normalize(values.min(), values.max())
colors = plt.cm.jet(norm(values))
for rec, col in zip(ax.patches, colors):
    rec.set_color(col)
plt.xlabel("Histogram of non-zero weights", size=14)
plt.show();
del values
gc.collect();  #weights分布圖0.05~0.2感覺最多''' #weughts分布圖

In [ ]:
'''train_data['weight_resp']   = train_data['weight']*train_data['resp']
train_data['weight_resp_1'] = train_data['weight']*train_data['resp_1']
train_data['weight_resp_2'] = train_data['weight']*train_data['resp_2']
train_data['weight_resp_3'] = train_data['weight']*train_data['resp_3']
train_data['weight_resp_4'] = train_data['weight']*train_data['resp_4']

fig, ax = plt.subplots(figsize=(15, 5))
resp    = pd.Series(1+(train_data.groupby('date')['weight_resp'].mean())).cumprod()
resp_1  = pd.Series(1+(train_data.groupby('date')['weight_resp_1'].mean())).cumprod()
resp_2  = pd.Series(1+(train_data.groupby('date')['weight_resp_2'].mean())).cumprod()
resp_3  = pd.Series(1+(train_data.groupby('date')['weight_resp_3'].mean())).cumprod()
resp_4  = pd.Series(1+(train_data.groupby('date')['weight_resp_4'].mean())).cumprod()
ax.set_xlabel ("Day", fontsize=18)
ax.set_title ("Cumulative daily return for resp and time horizons 1, 2, 3, and 4 (500 days)", fontsize=18)
resp.plot(lw=3, label='resp x weight')
resp_1.plot(lw=3, label='resp_1 x weight')
resp_2.plot(lw=3, label='resp_2 x weight')
resp_3.plot(lw=3, label='resp_3 x weight')
resp_4.plot(lw=3, label='resp_4 x weight')
# day 85 marker
ax.axvline(x=85, linestyle='--', alpha=0.3, c='red', lw=1)
ax.axvspan(0, 85 , color=sns.xkcd_rgb['grey'], alpha=0.1)
plt.legend(loc="lower left");  '''
#weight * resp的累積值
#紫色收益最多 投資越久收益越大 但風險越高

In [ ]:
#feature 有130個匿名特徵 其中feature_0為1與-1
train_data['feature_0'].value_counts()

In [ ]:
'''fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2,figsize=(20,10))

ax1.plot((pd.Series(train_data['feature_1']).cumsum()), lw=3, color='red')
ax1.set_title ("Linear", fontsize=22);
ax1.axvline(x=514052, linestyle='--', alpha=0.3, c='green', lw=2)
ax1.axvspan(0, 514052 , color=sns.xkcd_rgb['grey'], alpha=0.1)
ax1.set_xlim(xmin=0)
ax1.set_ylabel ("feature_1", fontsize=18);

ax2.plot((pd.Series(train_data['feature_3']).cumsum()), lw=3, color='green')
ax2.set_title ("Noisy", fontsize=22);
ax2.axvline(x=514052, linestyle='--', alpha=0.3, c='red', lw=2)
ax2.axvspan(0, 514052 , color=sns.xkcd_rgb['grey'], alpha=0.1)
ax2.set_xlim(xmin=0)
ax2.set_ylabel ("feature_3", fontsize=18);

ax3.plot((pd.Series(train_data['feature_55']).cumsum()), lw=3, color='darkorange')
ax3.set_title ("Hybryd (Tag 21)", fontsize=22);
ax3.set_xlabel ("Trade", fontsize=18)
ax3.axvline(x=514052, linestyle='--', alpha=0.3, c='green', lw=2)
ax3.axvspan(0, 514052 , color=sns.xkcd_rgb['grey'], alpha=0.1)
ax3.set_xlim(xmin=0)
ax3.set_ylabel ("feature_55", fontsize=18);

ax4.plot((pd.Series(train_data['feature_73']).cumsum()), lw=3, color='blue')
ax4.set_title ("Negative", fontsize=22)
ax4.set_xlabel ("Trade", fontsize=18)
ax4.set_ylabel ("feature_73", fontsize=18);
gc.collect();'''

#四種特徵類型 拿出代表feature1.3.55.73

In [ ]:
'''sns.set_palette("bright")

fig, axes = plt.subplots(2,2,figsize=(8,8))

sns.distplot(train_data[['feature_60']], hist=True, bins=200,  ax=axes[0,0])
sns.distplot(train_data[['feature_61']], hist=True, bins=200,  ax=axes[0,0])
axes[0,0].set_title ("features 60 and 61", fontsize=18)
axes[0,0].legend(labels=['60', '61'])

sns.distplot(train_data[['feature_62']], hist=True,  bins=200, ax=axes[0,1])
sns.distplot(train_data[['feature_63']], hist=True,  bins=200, ax=axes[0,1])
axes[0,1].set_title ("features 62 and 63", fontsize=18)
axes[0,1].legend(labels=['62', '63'])

sns.distplot(train_data[['feature_65']], hist=True,  bins=200, ax=axes[1,0])
sns.distplot(train_data[['feature_66']], hist=True,  bins=200, ax=axes[1,0])
axes[1,0].set_title ("features 65 and 66", fontsize=18)
axes[1,0].legend(labels=['65', '66'])


sns.distplot(train_data[['feature_67']], hist=True,  bins=200, ax=axes[1,1])
sns.distplot(train_data[['feature_68']], hist=True,  bins=200, ax=axes[1,1])
axes[1,1].set_title ("features 67 and 68", fontsize=18)
axes[1,1].legend(labels=['67', '68'])

plt.show();
gc.collect();'''

#可以看出60.61 and 62.63 and 65.66 and 67.68相關度很高 
#後面特徵工程會嘗試使用方法 減少特徵

In [ ]:
#del train_data, train_data_datatable#

In [ ]:
'''import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', 500)


# Standard plotly imports
import plotly as py
import plotly.graph_objs as go
import plotly.tools as tls
from plotly.offline import iplot, init_notebook_mode
import cufflinks
import cufflinks as cf
import plotly.figure_factory as ff
import os
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

import os


from sklearn import preprocessing
import xgboost as xgb'''

In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', 500)

import cufflinks
import cufflinks as cf

from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

import os


from sklearn import preprocessing
import xgboost as xgb

In [ ]:
#%%time
#%%time
#train = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
#載入太久

# install dabl
#!pip install dabl > /dev/null
#import dabl
# install datatable
#!pip install datatable > /dev/null
#import datatable as dt

In [ ]:
%%time
#利用fread載入data

train_data_datatable = dt.fread('../input/jane-street-market-prediction/train.csv')

#再轉為pandas方式  載入時間縮短很多！！ 直接載入pandas要2分多

train_data = train_data_datatable.to_pandas()

In [ ]:
import janestreet
env = janestreet.make_env()
iter_test = env.iter_test()

In [ ]:
train = train_data[train_data['weight'] != 0]
train['action'] = ((train['weight'].values * train['resp'].values) > 0).astype('int')#true為1 false為0

In [ ]:
X_train = train.loc[:, train.columns.str.contains('feature')] #取出包含feature標籤0~129
#print(X_train)
y_train = train.loc[:, 'action']
#print(y_train)
#y_train.isnull().any()

In [ ]:
del train

In [ ]:
X_train = X_train.fillna(-999)

In [ ]:
from sklearn import feature_selection as fs
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV

In [ ]:
sampler = TPESampler(seed=666)

def create_model(trial):
    max_depth = trial.suggest_int("max_depth", 2, 12)
    n_estimators = trial.suggest_int("n_estimators", 2, 600)
    learning_rate = trial.suggest_uniform('learning_rate', 0.0001, 0.99)
    subsample = trial.suggest_uniform('subsample', 0.0001, 1.0)
    colsample_bytree = trial.suggest_uniform('colsample_bytree', 0.0000001, 1)
    
    model = XGBClassifier(
        n_estimators=n_estimators, 
        max_depth=max_depth, 
        learning_rate=learning_rate,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        random_state=666,
        tree_method='gpu_hist'
    )
    return model

def objective(trial):
    model = create_model(trial)
    model.fit(X_train, y_train)
    score = accuracy_score(
        y_train, 
        model.predict(X_train)
    )
    return score

# study = optuna.create_study(direction="maximize", sampler=sampler)
# study.optimize(objective, n_trials=70)
# params = study.best_params
# params['random_state'] = 666
# params['tree_method'] = 'gpu_hist'

params1 = {
    'max_depth': 8, 
    'n_estimators': 500, 
    'learning_rate': 0.01, 
    'subsample': 0.9, 
    'tree_method': 'gpu_hist',
    'random_state': 666
}

# params2 = {
#     'max_depth': 9, 
#     'n_estimators': 500, 
#     'learning_rate': 0.03, 
#     'subsample': 0.9, 
#     'tree_method': 'gpu_hist',
#     'random_state': 666
# }

params3 = {
    'max_depth': 10, 
    'n_estimators': 500, 
    'learning_rate': 0.03, 
    'subsample': 0.9, 
    'colsample_bytree': 0.7,
    'tree_method': 'gpu_hist',
    'random_state': 666
}

In [ ]:
model1 = XGBClassifier(**params1)
model1.fit(X_train, y_train)

# model2 = XGBClassifier(**params2)
# model2.fit(X_train, y_train)

model3 = XGBClassifier(**params3)
model3.fit(X_train, y_train)

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    if test_df['weight'].item() > 0:
        X_test = test_df.loc[:, test_df.columns.str.contains('feature')]
        X_test = X_test.fillna(-999)
        # y_preds = model1.predict(X_test) + model2.predict(X_test) + model3.predict(X_test)
        y_preds = model1.predict(X_test) + model3.predict(X_test)
        if y_preds == 2:
            y_preds = np.array([1])
        else:
            y_preds = np.array([0])
    else:
        y_preds = np.array([0])
    sample_prediction_df.action = y_preds
    env.predict(sample_prediction_df)

In [ ]:
'''min_features_to_select = 1  # Minimum number of features to consider
rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(2),
              scoring='accuracy',
              min_features_to_select=min_features_to_select)
rfecv.fit(X_train, y_train)'''

In [ ]:
#特徵  上面提到60.61 and 62.63 and 65.66 and 67.68 相關度很高 在這裡處理
#weight 為0先拿掉
'''print('data Len:',len(train_data))
train_data = train_data[train_data['weight'] != 0]
print('data Len-weight=0:',len(train_data))
train_data['action'] = ((train_data['weight'].values * train_data['resp'].values) > 0).astype('int')
#print(train_data['action'])
#weeight*resp 新增欄位設為要的答案
X_train = train_data.loc[:, train_data.columns.str.contains('feature')]
#print(X_train)
y_train = train_data.loc[:, 'action']
#print(y_train)
#get X,y
X_train = X_train.fillna(-999) #缺失值填補-999'''

In [ ]:
'''from sklearn.decomposition import PCA  #PCA
from sklearn import datasets
import seaborn as sns
# 執行PCA之前，我們需要對特徵進行歸一化，以使其均值和單位方差為零
scaler = StandardScaler()
scaler.fit(X_train)
X_train_norm = scaler.transform(X_train)

pca = PCA()
comp = pca.fit(X_train_norm)
'''
#繪製圖表以顯示129個特徵中解釋的變化如何隨主要成分的數量而變化
#plt.plot(np.cumsum(comp.explained_variance_ratio_))
#plt.grid()
#plt.xlabel('Number of Principal Components')
#plt.ylabel('Explained Variance')
#sns.despine();

#前15個主要成分解釋了大約80％的變化
#前40個主要成分解釋了大約95％的變化
#pca = PCA(n_components=50).fit(X_train_norm)     #維度出使用50個features
#X_train = pca.transform(X_train_norm)

In [ ]:
#del pca, comp, X_train_norm, scaler

In [ ]:
'''clf = xgb.XGBClassifier(
    n_estimators=400,
    max_depth=7,
    eta=0.5, # learning_rate
    missing=None,
    random_state=42,
    tree_method='gpu_hist',
    subsample=0.8,
    colsample_bytree=1,
    #sampling_method='gradient_based',
    #eval_metric='logloss',
    verbosity=2   # info
)'''

In [ ]:
#%%time
#clf.fit(X_train, y_train)

In [ ]:
#clf.save_model('test.model')

In [ ]:
#iter_test = X_teest 測試用

In [ ]:
'''def fillna_npwhere(array, values):
    if np.isnan(array.sum()):
        array = np.where(np.isnan(array), values, array)
    return array'''

In [ ]:
'''rcount = 0
for (test_df, prediction_df) in env.iter_test():
    X_test = test_df.loc[:, features]
    y_preds = clf.predict(X_test)
    prediction_df.action = y_preds
    env.predict(prediction_df)
    rcount += len(test_df.index)'''

In [ ]:
'''for (test_df, sample_prediction_df) in iter_test:
    X_test = test_df.loc[:, test_df.columns.str.contains('feature')]
    X_test = X_test.fillna(-999)
    
    scaler = StandardScaler()
    scaler.fit(X_test)
    X_test_norm = scaler.transform(X_test)

    #comp = pca.fit(X_test_norm)
    #pca = PCA(n_components=50).fit(X_test_norm)     #維度出使用50個features
    X_train = pca.transform(X_test_norm)
    
    
    
    y_preds = clf.predict(X_test)
    sample_prediction_df.action = y_preds
    submission=env.predict(sample_prediction_df)'''

In [ ]:
'''#for (test_df, sample_prediction_df) in iter_test:
X_test = test_df.loc[:, test_df.columns.str.contains('feature')]
X_test = X_test.fillna(-999)
print(X_test)
X_test = Feature_PCA(X_test)
y_preds = clf.predict(X_test)  #測試使用 (原本使用defPCA發現不行這樣用)
sample_prediction_df.action = y_preds
submission=env.predict(sample_prediction_df)'''

In [ ]:
'''X_test = X_test.loc[:, X_test.columns.str.contains('feature')]
X_test = X_test.fillna(-999)

X_test = Feature_PCA(X_test)

y_preds = clf.predict(X_test)

sample_prediction_df.action = y_preds
env.predict(sample_prediction_df)'''

In [ ]:
submission=pd.read_csv('../input/jane-street-market-prediction/example_sample_submission.csv',sep=',')

submission.to_csv('./submission.csv',index=False)